<a href="https://colab.research.google.com/github/zainazeem41472/Rice-Leaf-Disease-Classification/blob/main/customcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
DATASET_DIR = "/content/drive/MyDrive/RiceLeafsDisease"
TRAIN_DIR = DATASET_DIR + "/train"
VAL_DIR = DATASET_DIR + "/validation"


In [3]:
!ls "$DATASET_DIR"
!ls "$TRAIN_DIR"
!ls "$VAL_DIR"


train  validation
bacterial_leaf_blight  healthy	   leaf_scald
brown_spot	       leaf_blast  narrow_brown_spot
bacterial_leaf_blight  healthy	   leaf_scald
brown_spot	       leaf_blast  narrow_brown_spot


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = 224
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

val_gen = val_datagen.flow_from_directory(
    VAL_DIR,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

num_classes = train_gen.num_classes
print("Classes:", num_classes)


Found 2100 images belonging to 6 classes.
Found 528 images belonging to 6 classes.
Classes: 6


In [5]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD

# Load VGG16 without top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

# Freeze pretrained layers
for layer in base_model.layers:
    layer.trainable = False

# Classifier head
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)

# Build model
model = Model(inputs=base_model.input, outputs=output)

# -------- NEW OPTIMIZER, LOSS, METRICS --------
model.compile(
    optimizer=SGD(learning_rate=0.001, momentum=0.9),  # New optimizer
    loss='sparse_categorical_crossentropy',            # New loss
    metrics=['precision', 'recall']                    # New metrics
)

model.summary()


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     6,422,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │         1,542 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,139,014 (80.64 MB)

 Trainable params: 6,424,326 (24.51 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [6]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD

# Swish activation
def swish(x):
    from tensorflow.keras.activations import sigmoid
    return x * sigmoid(x)

inputs = Input(shape=(img_size, img_size, 3))

# ---------------- BLOCK 1 ----------------
x = Conv2D(64, (3,3), activation=swish, padding='same')(inputs)
x = Conv2D(64, (3,3), activation=swish, padding='same')(x)
x = MaxPooling2D((2,2))(x)

# ---------------- BLOCK 2 ----------------
x = Conv2D(128, (3,3), activation=swish, padding='same')(x)
x = Conv2D(128, (3,3), activation=swish, padding='same')(x)
x = MaxPooling2D((2,2))(x)

# ---------------- BLOCK 3 ----------------
x = Conv2D(256, (3,3), activation=swish, padding='same')(x)
x = Conv2D(256, (3,3), activation=swish, padding='same')(x)
x = Conv2D(256, (3,3), activation=swish, padding='same')(x)
x = MaxPooling2D((2,2))(x)

# ---------------- BLOCK 4 ----------------
x = Conv2D(512, (3,3), activation=swish, padding='same')(x)
x = Conv2D(512, (3,3), activation=swish, padding='same')(x)
x = Conv2D(512, (3,3), activation=swish, padding='same')(x)
x = MaxPooling2D((2,2))(x)

# ---------------- BLOCK 5 ----------------
x = Conv2D(512, (3,3), activation=swish, padding='same')(x)
x = Conv2D(512, (3,3), activation=swish, padding='same')(x)
x = Conv2D(512, (3,3), activation=swish, padding='same')(x)
x = MaxPooling2D((2,2))(x)

# -------- CLASSIFIER HEAD --------
x = Flatten()(x)
x = Dense(4096, activation=swish)(x)
x = Dropout(0.5)(x)
x = Dense(4096, activation=swish)(x)
x = Dropout(0.5)(x)

output = Dense(num_classes, activation='softmax')(x)

# Build model
model = Model(inputs, output)

# -------- LOSS = Mean Squared Error --------
model.compile(
    optimizer=SGD(learning_rate=0.001, momentum=0.9),
    loss='mean_squared_error',
    metrics=['accuracy']
)

model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 6)              │        24,58

 Total params: 134,285,126 (512.26 MB)

 Trainable params: 134,285,126 (512.26 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
epochs = 100

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=epochs
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 1020s 15s/step - accuracy: 0.1562 - loss: 0.1389 - val_accuracy: 0.1667 - val_loss: 0.1389
Epoch 2/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - accuracy: 0.1695 - loss: 0.1389 - val_accuracy: 0.1667 - val_loss: 0.1389
Epoch 3/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 70s 1s/step - accuracy: 0.1538 - loss: 0.1389 - val_accuracy: 0.1667 - val_loss: 0.1389
Epoch 4/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - accuracy: 0.1633 - loss: 0.1389 - val_accuracy: 0.1667 - val_loss: 0.1389
Epoch 5/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - accuracy: 0.1602 - loss: 0.1389 - val_accuracy: 0.1667 - val_loss: 0.1389
Epoch 6/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step - accuracy: 0.1628 - loss: 0.1389 - val_accuracy: 0.1667 - val_loss: 0.1389
Epoch 7/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 70s 1s/step - accuracy: 0.1605 - loss: 0.1389 - val_accuracy: 0.1667 - val_loss: 0.1389
Epoch 8/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - accuracy: 0.1670 - loss: 0.1389 - val_accuracy: 0.1667 

In [9]:
final_accuracy = history.history['accuracy'][-1] * 100
print("Final Training Accuracy: {:.2f}%".format(final_accuracy))


Final Training Accuracy: 15.43%
